In [1]:
import pypsa
import pandas as pd
import numpy as np

In [43]:
n = pypsa.Network("elec_s_4.nc")

INFO:pypsa.io:Imported network elec_s_4.nc has buses, carriers, generators, lines, links, loads


In [44]:
n.import_from_csv_folder("Export_Import")

ERROR:pypsa.io:Error, new components for Bus are not unique
ERROR:pypsa.io:Error, new components for Carrier are not unique
ERROR:pypsa.io:Error, new components for Generator are not unique
ERROR:pypsa.io:Error, new components for Line are not unique
ERROR:pypsa.io:Error, new components for Link are not unique
ERROR:pypsa.io:Error, new components for Load are not unique
INFO:pypsa.io:Imported network Export_Import has buses, carriers, generators, lines, links, loads


In [45]:
#Kosten von Solar und Onwind auf 0 setzen
#Schleife hier schwer, da n.generators keine wiederkehrende Form hat
n.generators.loc['DE0 0 onwind','marginal_cost']=0
n.generators.loc['DE0 1 onwind','marginal_cost']=0
n.generators.loc['DE0 2 onwind','marginal_cost']=0
n.generators.loc['DE0 3 onwind','marginal_cost']=0
n.generators.loc['DE0 0 solar','marginal_cost']=0
n.generators.loc['DE0 1 solar','marginal_cost']=0
n.generators.loc['DE0 2 solar','marginal_cost']=0
n.generators.loc['DE0 3 solar','marginal_cost']=0
n.generators.loc['DE0 0 onwind','capital_cost']=0
n.generators.loc['DE0 1 onwind','capital_cost']=0
n.generators.loc['DE0 2 onwind','capital_cost']=0
n.generators.loc['DE0 3 onwind','capital_cost']=0
n.generators.loc['DE0 0 solar','capital_cost']=0
n.generators.loc['DE0 1 solar','capital_cost']=0
n.generators.loc['DE0 2 solar','capital_cost']=0
n.generators.loc['DE0 3 solar','capital_cost']=0

In [46]:
#Da es beim Einlesen der CSV's nur Werte hinzugefügt werden und nicht entfernt werden können
#-->werfe hier die Offwind Spalten aus generators.p_max_pu raus
n.generators_t.p_max_pu = n.generators_t.p_max_pu[n.generators_t.p_max_pu.columns.drop(list(n.generators_t.p_max_pu.filter(regex='offwind')))]

In [47]:
#Bringe nun die max_pu_Daten in ein Intervall zwischen 0 und 1 und belege p_nom entsprechend,
#Belege außerdem p_nom_max mit dem entsprechenden Wert, sodass keine Überproduktion stattfindet

#Erhalte die Maxima der einzelnen Onwind und Solar Daten
nodes=4 #Anzahl Knoten
max_pu_onwind = np.zeros((1,nodes),dtype=float) #Initialisierung
max_pu_solar = np.zeros((1,nodes),dtype=float) #Initialisierung
for i in range(0,nodes):
    max_pu_onwind[0,i] = max(n.generators_t.p_max_pu.iloc[:,2*i])
    max_pu_solar[0,i] = max(n.generators_t.p_max_pu.iloc[:,2*i+1])
    
#Dividiere jeden Wert durch das Maximum um im Intervall 0 und 1 zu landen
for i in range(0,nodes):
    n.generators_t.p_max_pu.iloc[:,i*2]=n.generators_t.p_max_pu.iloc[:,i*2] / max_pu_onwind[0,i]
    n.generators_t.p_max_pu.iloc[:,i*2+1]=n.generators_t.p_max_pu.iloc[:,i*2+1] / max_pu_solar[0,i]

    
#Setze die vorherigen Maxima nun auf p_nom, respektive der jeweiligen Generatoren
#(nicht mit Schleife lösbar, da Daten in Generators keine feste Form haben)
n.generators.loc["DE0 0 onwind","p_nom"] = max_pu_onwind[0,0]
n.generators.loc["DE0 1 onwind","p_nom"] = max_pu_onwind[0,1]
n.generators.loc["DE0 2 onwind","p_nom"] = max_pu_onwind[0,2]
n.generators.loc["DE0 3 onwind","p_nom"] = max_pu_onwind[0,3]
n.generators.loc["DE0 0 solar","p_nom"] = max_pu_solar[0,0]
n.generators.loc["DE0 1 solar","p_nom"] = max_pu_solar[0,1]
n.generators.loc["DE0 2 solar","p_nom"] = max_pu_solar[0,2]
n.generators.loc["DE0 3 solar","p_nom"] = max_pu_solar[0,3]

#belege p_nom_max mit den ausgerechneten p_nom Werten
n.generators.loc["DE0 0 onwind","p_nom_max"] = max_pu_onwind[0,0]
n.generators.loc["DE0 1 onwind","p_nom_max"] = max_pu_onwind[0,1]
n.generators.loc["DE0 2 onwind","p_nom_max"] = max_pu_onwind[0,2]
n.generators.loc["DE0 3 onwind","p_nom_max"] = max_pu_onwind[0,3]
n.generators.loc["DE0 0 solar","p_nom_max"] = max_pu_solar[0,0]
n.generators.loc["DE0 1 solar","p_nom_max"] = max_pu_solar[0,1]
n.generators.loc["DE0 2 solar","p_nom_max"] = max_pu_solar[0,2]
n.generators.loc["DE0 3 solar","p_nom_max"] = max_pu_solar[0,3]

In [48]:
#Verhindere, dass andere Generatoren außer Kohle, Wind und Solar Energie erzeugen können
n.generators.loc["DE0 0 CCGT","p_nom_max"] = 0
n.generators.loc["DE0 1 CCGT","p_nom_max"] = 0
n.generators.loc["DE0 2 CCGT","p_nom_max"] = 0
n.generators.loc["DE0 3 CCGT","p_nom_max"] = 0

n.generators.loc["DE0 0 OCGT","p_nom_max"] = 0
n.generators.loc["DE0 1 OCGT","p_nom_max"] = 0
n.generators.loc["DE0 2 OCGT","p_nom_max"] = 0
n.generators.loc["DE0 3 OCGT","p_nom_max"] = 0

n.generators.loc["DE0 0 offwind-ac","p_nom_max"] = 0
n.generators.loc["DE0 0 offwind-dc","p_nom_max"] = 0
n.generators.loc["DE0 1 offwind-ac","p_nom_max"] = 0
n.generators.loc["DE0 1 offwind-dc","p_nom_max"] = 0

#n.generators.loc["DE0 0 coal","p_nom_max"] = 0
#n.generators.loc["DE0 1 coal","p_nom_max"] = 0
#n.generators.loc["DE0 2 coal","p_nom_max"] = 0
#n.generators.loc["DE0 3 coal","p_nom_max"] = 0

In [49]:
#Setze die Kosten von Kohle alle gleich
n.generators.loc["DE0 0 coal","marginal_cost"] = 20000
n.generators.loc["DE0 1 coal","marginal_cost"] = 20000
n.generators.loc["DE0 2 coal","marginal_cost"] = 20000
n.generators.loc["DE0 3 coal","marginal_cost"] = 20000

n.generators.loc["DE0 0 coal","p_nom"] = 0
n.generators.loc["DE0 1 coal","p_nom"] = 0
n.generators.loc["DE0 2 coal","p_nom"] = 0
n.generators.loc["DE0 3 coal","p_nom"] = 0

n.generators.loc["DE0 0 coal","capital_cost"] = 20000
n.generators.loc["DE0 1 coal","capital_cost"] = 20000
n.generators.loc["DE0 2 coal","capital_cost"] = 20000
n.generators.loc["DE0 3 coal","capital_cost"] = 20000

In [50]:
#Es muss nichts über die Lines und Links geschickt werden
n.lines.loc[:,"s_nom_extendable"] = True
#n.links.loc[:,"p_nom_extendable"] = True

In [51]:
#p_nom jedes Generators soll extendable sein. Mit dem p_nom_max im vorherigen Schritt und
#den Kosten von 0 erreichen wir unser Ziel
n.generators.loc[:,"p_nom_extendable"]=True

In [52]:
#Setze die Kapazität der Lines auf 0, damit jeglicher Strom der drüber geschickt wird etwas kostet.
n.lines.loc["1","s_nom"] = 0
n.lines.loc["2","s_nom"] = 0
n.lines.loc["3","s_nom"] = 0
n.lines.loc["4","s_nom"] = 0
n.lines.loc["5","s_nom"] = 0

In [53]:
#Setting Capital Costs of Energy Transfer high to ensure local production of energy
n.lines.loc["1","capital_cost"] = 20000
n.lines.loc["2","capital_cost"] = 20000
n.lines.loc["3","capital_cost"] = 20000
n.lines.loc["4","capital_cost"] = 20000
n.lines.loc["5","capital_cost"] = 20000

In [54]:
n.lopf()

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4155037726325.82
  Upper bound: 4155037726325.82
  Number of objectives: 1
  Number of constraints: 52081
  Number of variables: 23838
  Number of nonzeros: 105747
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 6.377948522567749
# ----------------------------------------------------------
#   Solution Information
# ----

(<SolverStatus.ok: 'ok'>, <TerminationCondition.optimal: 'optimal'>)

In [55]:
n.export_to_csv_folder('Solution')

INFO:pypsa.io:Exported network Solution has buses, links, generators, carriers, loads, lines


,p_nom_max,weight,p_nom,capital_cost,p_nom_extendable,carrier,efficiency,marginal_cost,bus,control,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
name,,,,,,,,,,,,,,,,,,,,,
DE0 0 CCGT,0.0,0.000000,2827.810000,0.000000,True,CCGT,0.500,4.720000e+01,DE0 0,Slack,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
DE0 0 OCGT,0.0,0.000000,0.000000,47234.561404,True,OCGT,0.390,5.838462e+01,DE0 0,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
DE0 0 coal,inf,0.000000,5927.300000,0.000000,True,coal,0.464,9.999999e+06,DE0 0,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
DE0 0 offwind-ac,0.0,10985.663855,0.000000,203745.308800,True,offwind-ac,1.000,1.500000e-02,DE0 0,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
DE0 0 offwind-dc,0.0,1811.127821,0.000000,222161.027801,True,offwind-dc,1.000,1.500000e-02,DE0 0,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
DE0 0 onwind,30652.0,46170.006716,30652.000000,0.000000,True,onwind,1.000,0.000000e+00,DE0 0,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
DE0 0 solar,17627.0,16450.821580,17627.000000,0.000000,True,solar,1.000,0.000000e+00,DE0 0,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
DE0 1 CCGT,0.0,0.000000,9138.300000,0.000000,True,CCGT,0.500,4.720000e+01,DE0 1,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
DE0 1 OCGT,0.0,0.000000,0.000000,47234.561404,True,OCGT,0.390,5.838462e+01,DE0 1,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
